In [ ]:
# Load necessary libraries
library(dplyr)
library(Matrix)
library(data.table)
library(Seurat)
library(ggplot2)
library(RColorBrewer)
library(cowplot)
library(tidyverse)
library(tidyr)
library(viridis)
library(harmony)
library(MASS) #
library(scales)
library(AnnotationHub)	
library(org.Hs.eg.db)  
library(clusterProfiler)
library(biomaRt)
library(ggthemes)
library(msigdbr)
library(conflicted)
library(stringr)


In [ ]:
###

# Read the Seurat object for Micro Astro
scRNA1 = readRDS('./')

# Set the default assay to RNA
DefaultAssay(scRNA1) = "RNA"

# Data preprocessing
scRNA1 = SCTransform(scRNA1)
scRNA1 = NormalizeData(scRNA1)
scRNA1 = ScaleData(scRNA1)
scRNA1 = FindVariableFeatures(scRNA1, selection.method = "vst")

# Run PCA
scRNA1 <- RunPCA(scRNA1, features = VariableFeatures(scRNA1))

# Elbow plot to determine the number of significant PCs
plot2 <- ElbowPlot(scRNA1, ndims = 40, reduction = "pca")

# Define the number of PCs to use
pc.num = 1:20

# Run Harmony integration
scRNA1 = RunHarmony(scRNA1, group.by.vars = "sample", dims.use = pc.num, max.iter.harmony = 20)

# Find neighbors and clusters
scRNA1 <- FindNeighbors(scRNA1, reduction = "harmony", dims = pc.num)
scRNA1 <- FindClusters(scRNA1, resolution = c(0.1, 0.5, 0.8, 1, 1.2, 1.5))

# Set default assay to SCT
DefaultAssay(scRNA1) = "SCT"

# Run UMAP for dimensionality reduction
scRNA1 = RunUMAP(scRNA1, reduction = "harmony", dims = 1:25, n.neighbors = 50, min.dist = 0.4)

# Set the identity class
Idents(scRNA1) = 'SCT_snn_res.0.6'

# Feature plot for the 'DAM1' marker
p = FeaturePlot(scRNA1, 'DAM1', reduction = 'umap', cols = rev(brewer.pal(10, name = "RdBu")), raster = FALSE)

# Add a border to the plot
p + theme(panel.border = element_rect(fill = NA, color = "black", size = 0.01, linetype = "solid"))
